## LeNet


&emsp;&emsp;LeNet(LeNet-5)由两个部分组成: 卷积编码器和全连接层密集块。

In [1]:
import torch
import torch.nn as nn

class Reshape(nn.Module):
    def forward(self, x):
        return x.view(-1, 1, 28, 28) # 添加通道数。

net = torch.nn.Sequential(
    Reshape(), nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(),
    nn.AvgPool2d(2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.Sigmoid(),
    nn.Linear(84, 10)
)

In [2]:
X = torch.randn(size=(1, 1, 28, 28), dtype=torch.float32)
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, "Out Put Shape: ", X.shape)

Reshape Out Put Shape:  torch.Size([1, 1, 28, 28])
Conv2d Out Put Shape:  torch.Size([1, 6, 28, 28])
Sigmoid Out Put Shape:  torch.Size([1, 6, 28, 28])
AvgPool2d Out Put Shape:  torch.Size([1, 6, 14, 14])
Conv2d Out Put Shape:  torch.Size([1, 16, 10, 10])
Sigmoid Out Put Shape:  torch.Size([1, 16, 10, 10])
AvgPool2d Out Put Shape:  torch.Size([1, 16, 5, 5])
Flatten Out Put Shape:  torch.Size([1, 400])
Linear Out Put Shape:  torch.Size([1, 120])
Sigmoid Out Put Shape:  torch.Size([1, 120])
Linear Out Put Shape:  torch.Size([1, 84])
Sigmoid Out Put Shape:  torch.Size([1, 84])
Linear Out Put Shape:  torch.Size([1, 10])


## 加载数据集

In [3]:
import torchvision
from torchvision import transforms
from torch.utils import data

def get_dataloader_workers():
    """Use 4 processes to read the data."""
    return 4

def load_data_fashion_mnist(batch_size, resize=None):
    """Download the Fashion-MNIST dataset and then load it into memory."""
    trans = [transforms.ToTensor()]
    
    if resize:
        trans.insert(0, transforms.Resize(resize))
        
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root="../data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(root="../data", train=False, transform=trans, download=True)
    
    return (data.DataLoader(mnist_train, batch_size, shuffle=True, num_workers=get_dataloader_workers()),
            data.DataLoader(mnist_test, batch_size, shuffle=False, num_workers=get_dataloader_workers()))

In [4]:
train_iter, test_iter = load_data_fashion_mnist(batch_size=256)

/Users/tinyzqh/miniforge3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /tmp/pip-req-build-wdh4qigd/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## 使用GPU

&emsp;&emsp;定义一个累加器:

In [5]:
class Accumulator:
    """For accumulating sums over `n` variables."""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [6]:
argmax = lambda x, *args, **kwargs: x.argmax(*args, **kwargs)
reduce_sum = lambda x, *args, **kwargs: x.sum(*args, **kwargs)
astype = lambda x, *args, **kwargs: x.type(*args, **kwargs)

def accuracy(y_hat, y):
    """Compute the number of correct predictions."""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = argmax(y_hat, axis=1)
    cmp = astype(y_hat, y.dtype) == y
    return float(reduce_sum(astype(cmp, y.dtype)))

In [7]:
def evaluate_accuracy_gpu(net, data_iter, device=None):
    """使用GPU计算模型在数据集上的精度"""
    if isinstance(net, torch.nn.Module):
        net.eval()
        if not device: # 如果没有指定device, 就用网络参数中的所给定的那个device。
            device = next(iter(net.parameters())).device
    
    metric = Accumulator(2)
    
    for X, y in data_iter:
        if isinstance(X, list):
            X = [x.to(device) for x in X]  # 将数据挪到device上。
        else:
            X = X.to(device)
            
        y = y.to(device)
        
        metric.add(accuracy(net(X), y), y.numel())
        
        return metric[0] / metric[1]

In [21]:
def train(net, train_iter, test_iter, num_epochs, lr, device):
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    
    net.apply(init_weights)  # 对net里面的每一个parameter都去run一下init_weights这个函数。
    print("trainng on {}".format(device))
    
    net.to(device)
    
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    
    num_batches = len(train_iter)
    
    for epoch in range(num_epochs):
        metric = Accumulator(3)
        net.train()
        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            
            y_hat = net(X)
            
            optimizer.zero_grad()
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            
            metric.add(l * X.shape[0], accuracy(y_hat, y), X.shape[0])
            
            train_loss = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
            
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                print("epoch {}, train_loss {}, train_acc {}".format(epoch + (i + 1) / num_batches, 
                                                                  train_loss, train_acc))
    test_acc = evaluate_accuracy_gpu(net, test_iter)
    print(f'loss {train_loss:.3f}, train acc {train_acc:.3f}, 'f'test acc {test_acc:.3f}')

In [22]:
def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu()."""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

In [23]:
lr, num_epochs = 0.9, 10
train(net, train_iter, test_iter, num_epochs, lr, try_gpu())

trainng on cpu
epoch 0.2, train_loss 2.3627252680190067, train_acc 0.09898603723404255
epoch 0.4, train_loss 2.3387740799721253, train_acc 0.09832114361702128
epoch 0.6, train_loss 2.3296520321081715, train_acc 0.09801640070921985
epoch 0.8, train_loss 2.3235291458190757, train_acc 0.10114694148936171
epoch 1.0, train_loss 2.3051708170572915, train_acc 0.11141666666666666
epoch 1.2, train_loss 1.6401129529831258, train_acc 0.35555186170212766
epoch 1.4, train_loss 1.4330693825762322, train_acc 0.4287317154255319
epoch 1.6, train_loss 1.3167020391065178, train_acc 0.4711048315602837
epoch 1.8, train_loss 1.2337023615837097, train_acc 0.5032205784574468
epoch 2.0, train_loss 1.1681793467203776, train_acc 0.5301333333333333
epoch 2.2, train_loss 0.9025096728446635, train_acc 0.6457779255319149
epoch 2.4, train_loss 0.862120769759442, train_acc 0.6630236037234043
epoch 2.6, train_loss 0.8354263094300074, train_acc 0.6742298315602837
epoch 2.8, train_loss 0.8140013544483388, train_acc 0.681